In [1]:
import pandas as pd

In [2]:
no_of_kfolds = 10

#load_kfold_dfs,all_dfs
training_all_dfs = pd.DataFrame()
training_kfold_dfs = dict()
for kfold in range(1, no_of_kfolds+1):
    df = pd.read_csv('hybrid_recommender/training_data/combined_predictions_' + str(kfold) + '.csv')
    training_kfold_dfs[kfold] = df
    training_all_dfs = training_all_dfs.append(df)
    print(training_kfold_dfs[kfold].shape)
print(training_all_dfs.shape)
training_all_dfs.to_csv('hybrid_recommender/training_data/all_combined_predictions.csv', index=False)


testing_all_dfs = pd.DataFrame()
testing_kfold_dfs = dict()
for kfold in range(1, no_of_kfolds+1):
    df = pd.read_csv('hybrid_recommender/testing_data/combined_predictions_' + str(kfold) + '.csv')
    testing_kfold_dfs[kfold] = df
    testing_all_dfs = testing_all_dfs.append(df)
    print(testing_kfold_dfs[kfold].shape)
print(testing_all_dfs.shape)
testing_all_dfs.to_csv('hybrid_recommender/testing_data/all_combined_predictions.csv', index=False)

(8475, 10)
(8475, 10)
(8475, 10)
(8475, 10)
(8475, 10)
(8475, 10)
(8475, 10)
(8475, 10)
(8475, 10)
(8475, 10)
(84750, 10)
(4710, 10)
(4710, 10)
(4710, 10)
(4710, 10)
(4710, 10)
(4710, 10)
(4710, 10)
(4710, 10)
(4705, 10)
(4705, 10)
(47090, 10)


In [3]:
from functools import reduce
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from math import sqrt
import numpy as np

features = ['BaselineOnly_SGD_Tuned_est',
            'Knn_UserBased_ZScore_MSD_Tuned_est',
            'Knn_ItemBased_ZScore_MSD_Tuned_est',
            'Knn_UserBased_Baseline_SGD_Tuned_est',
            'Knn_ItemBased_Baseline_SGD_Tuned_est', 
            'SVD_biased_Tuned_est',
            'SVDpp_biased_Tuned_est']
target = 'r_ui'

In [21]:
def get_prediction_metrics(y_true, y_pred):
    metrics = dict()
    metrics['r2_score'] = r2_score(y_true, y_pred)
    metrics['mean_absolute_error'] = mean_absolute_error(y_true, y_pred)
    metrics['rmse'] = sqrt(mean_squared_error(y_true, y_pred))
    return metrics

def get_avg_test_metrics(kfold_test_metrics):
    metrics = dict()
    metrics['r2_score'] = []
    metrics['mean_absolute_error'] = []
    metrics['rmse'] = []
    
    for test_metrics in kfold_test_metrics:
        metrics['r2_score'].append(test_metrics['r2_score'])
        metrics['mean_absolute_error'].append(test_metrics['mean_absolute_error'])
        metrics['rmse'].append(test_metrics['rmse'])
    #print(metrics)
    avg_metrics = dict()
    avg_metrics['r2_score'] = round(np.mean(metrics['r2_score']), 4)
    avg_metrics['mean_absolute_error'] = round(np.mean(metrics['mean_absolute_error']), 4)
    avg_metrics['rmse'] = round(np.mean(metrics['rmse']), 4)
    return avg_metrics

def get_avg_metrics(estimator, kfold_dfs):
    print(estimator)
    kfold_test_metrics = []
    for kfold in range(1, no_of_kfolds+1):
        #print('*'*40)
        #print('kfold_', str(kfold))
        train_df = pd.DataFrame()
        test_df = pd.DataFrame()
        for kfold_id, df in kfold_dfs.items():
            if kfold == kfold_id:
                test_df = df
            else:
                train_df = train_df.append(df)
        #print("train_df : {} test_df : {}".format(train_df.shape, test_df.shape))
        train_users = set(train_df['uid'].unique())
        test_users = set(test_df['uid'].unique())
        common_users = train_users & test_users
        #print("No of train_users  : ", len(train_users))
        #print("No of test_users   : ", len(test_users))
        #print("No of common_users : ", len(common_users))

        train_X, train_y = train_df[features], train_df[target]    
        #print(train_X.shape, train_y.shape)
        test_X, test_y = test_df[features], test_df[target]
        #print(test_X.shape, test_y.shape)

        model = estimator.fit(train_X, train_y)

        #predicted_train_y = model.predict(train_X)
        #train_metrics = get_prediction_metrics(train_y, predicted_train_y)
        #print(train_metrics)
        predicted_test_y = model.predict(test_X)
        test_metrics = get_prediction_metrics(test_y, predicted_test_y)
        #print(test_metrics)
        kfold_test_metrics.append(test_metrics)
    #print(kfold_test_metrics)
    avg_metrics = get_avg_test_metrics(kfold_test_metrics)
    print('r2 : {:05.4f}, mae : {:05.4f}, rmse : {:05.4f}'.format(avg_metrics['r2_score'], 
                                                     avg_metrics['mean_absolute_error'],
                                                     avg_metrics['rmse']))
    #print(avg_metrics)
    print('*'*40)
    return avg_metrics

In [23]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

#Knn_UserBased_Baseline_SGD_Tuned                                      0.6948
#                                                                      0.6946

print("validation data")
get_avg_metrics(LinearRegression(), training_kfold_dfs)       #'rmse': 0.6910
print("testing data")
get_avg_metrics(LinearRegression(), testing_kfold_dfs)        #'rmse': 0.6912
print()

# print("validation data")
# get_avg_metrics(Ridge(), training_kfold_dfs)                #'rmse': 0.6910
# print("testing data")
# get_avg_metrics(Ridge(), testing_kfold_dfs)                 #'rmse': 0.6912
# print()

print("validation data")
get_avg_metrics(Ridge(alpha=10), training_kfold_dfs)          #'rmse': 0.6910
print("testing data")
get_avg_metrics(Ridge(alpha=10), testing_kfold_dfs)           #'rmse': 0.6912
print()


# print("validation data")
# get_avg_metrics(Lasso(), training_kfold_dfs)                #'rmse': 0.8229
# print("testing data")
# get_avg_metrics(Lasso(), testing_kfold_dfs)                 #'rmse': 0.8219
# print()

print("validation data")
get_avg_metrics(Lasso(alpha=0.0001), training_kfold_dfs)      #'rmse': 0.6910
print("testing data")
get_avg_metrics(Lasso(alpha=0.0001), testing_kfold_dfs)       #'rmse': 0.6912
print()


#get_avg_metrics(ElasticNet(), training_kfold_dfs)
#get_avg_metrics(SGDRegressor(), training_kfold_dfs)

#get_avg_metrics(RandomForestRegressor(), training_kfold_dfs)
#get_avg_metrics(GradientBoostingRegressor(), training_kfold_dfs)
print()

validation data
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
r2 : 0.2935, mae : 0.5532, rmse : 0.6910
****************************************
testing data
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
r2 : 0.2630, mae : 0.5545, rmse : 0.6912
****************************************

validation data
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
r2 : 0.2935, mae : 0.5532, rmse : 0.6910
****************************************
testing data
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
r2 : 0.2630, mae : 0.5545, rmse : 0.6912
****************************************

validation data
Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
r2 : 0.2935, mae : 0.5532, rmse : 0.6910
************

In [6]:
# from sklearn.svm import SVR
# get_avg_metrics(SVR())